<a href="https://www.kaggle.com/code/shahrish99/cornet-s-cifar100?scriptVersionId=256937529" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Model Creation

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Check for GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters
BATCH_SIZE = 64
LR = 1e-3
NUM_EPOCHS = 10
STEP_SIZE = 5
GAMMA = 0.1

# Data preprocessing and augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.Resize(224),  # Resize CIFAR-100 images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, transform=transform_train, download=True)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, transform=transform_test, download=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Clone CORNet repository (specific to Google Colab)
!git clone https://github.com/dicarlolab/CORnet.git

# Navigate to the cloned repository folder
os.chdir('/kaggle/working/CORnet')

# Install the package
!pip install .

In [ ]:
# Import necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Specify the path to the saved checkpoint
checkpoint_path = '/kaggle/input/cornets-cifar100/pytorch/default/1/cornet_s_retrained_epoch15.pth'  # Update with the actual path after uploading

# Load the model
from cornet import cornet_s

model = cornet_s()

# Update the final layer for CIFAR-100
num_classes = 100
model.module.decoder.linear = nn.Linear(
    in_features=model.module.decoder.linear.in_features,
    out_features=num_classes
)
model = model.to(device)

# Load the saved state dictionary
model.load_state_dict(torch.load(checkpoint_path))
print(f"Model loaded from {checkpoint_path}")

In [ ]:
# Define loss function
criterion = nn.CrossEntropyLoss()

# Perform a validation loop to check accuracy
model.eval()
val_loss = 0.0
correct1 = 0
correct5 = 0
total = 0

print("Running validation to check initial accuracy...")
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Validation (Pre-training Check)"):
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * inputs.size(0)

        # Calculate top-1 and top-5 accuracy
        _, pred1 = outputs.topk(1, 1, True, True)
        _, pred5 = outputs.topk(5, 1, True, True)
        correct1 += pred1.eq(labels.view(-1, 1).expand_as(pred1)).sum().item()
        correct5 += pred5.eq(labels.view(-1, 1).expand_as(pred5)).sum().item()
        total += labels.size(0)

epoch_val_loss = val_loss / len(test_loader.dataset)
top1_acc = correct1 / total
top5_acc = correct5 / total

print(f"Validation Loss: {epoch_val_loss:.4f}")
print(f"Top-1 Accuracy: {top1_acc:.4f}, Top-5 Accuracy: {top5_acc:.4f}")

In [ ]:
# Hyperparameters
BATCH_SIZE = 64
LR = 1e-3  # Learning rate for re-training
NUM_EPOCHS = 15  # Continue training for more epochs
STEP_SIZE = 5
GAMMA = 0.5  # Slightly increase decay for better convergence

# Define loss function, optimizer, and learning rate scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

# Re-training loop
for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0

    with tqdm(train_loader, unit="batch", desc=f"Training Epoch {epoch+1}/{NUM_EPOCHS}") as tepoch:
        for inputs, labels in tepoch:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero accumulated gradients
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            tepoch.set_postfix(loss=(running_loss / len(train_loader.dataset)))

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} Training Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    val_loss = 0.0
    correct1 = 0
    correct5 = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc=f"Validation Epoch {epoch+1}/{NUM_EPOCHS}"):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            _, pred1 = outputs.topk(1, 1, True, True)
            _, pred5 = outputs.topk(5, 1, True, True)
            correct1 += pred1.eq(labels.view(-1, 1).expand_as(pred1)).sum().item()
            correct5 += pred5.eq(labels.view(-1, 1).expand_as(pred5)).sum().item()
            total += labels.size(0)

    epoch_val_loss = val_loss / len(test_loader.dataset)
    top1_acc = correct1 / total
    top5_acc = correct5 / total

    print(f"Validation Loss: {epoch_val_loss:.4f}")
    print(f"Top-1 Accuracy: {top1_acc:.4f}, Top-5 Accuracy: {top5_acc:.4f}")

    scheduler.step()

    # Save the model after each epoch
    torch.save(model.state_dict(), f'./cornet_s_retrained_epoch{epoch+1}.pth')

print("Re-training complete!")

In [ ]:
model

In [ ]:
!pip install torchattacks

In [ ]:
# List of attacks with their parameters
attacks = {
    "FGSM": torchattacks.FGSM(model, eps=0.3),
    "PGD": torchattacks.PGD(model, eps=0.3, alpha=2/255, steps=40),
    "CW": torchattacks.CW(model, c=1e-4, kappa=0, steps=1000, lr=0.01)
}

# Evaluate model on each attack
for attack_name, attack in attacks.items():
    correct = 0
    total = 0
    print(f"Running {attack_name} Attack...")
    
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        adv_inputs = attack(inputs, labels)  # Generate adversarial examples
        outputs = model(adv_inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print(f"{attack_name} Adversarial Accuracy: {100 * correct / total:.2f}%\n")

In [ ]:
from tqdm import tqdm
import torchattacks

# Ensure the model is on the correct device
model.to(device)

# Initialize the CW attack
cw_attack = torchattacks.CW(model, c=1e-4, kappa=0, steps=100, lr=0.01)

# Prepare to evaluate model under CW attack
cw_correct = 0
cw_total = 0

model.eval()  # Set the model to evaluation mode

# Iterate through the validation loader
for inputs, labels in tqdm(test_loader, desc="CW Attack Evaluation"):
    inputs, labels = inputs.to(device), labels.to(device)

    try:
        # Generate adversarial examples
        adv_inputs = cw_attack(inputs, labels)

        # Forward pass with adversarial examples
        outputs = model(adv_inputs)

        # Calculate accuracy
        _, predicted = outputs.max(1)
        cw_correct += predicted.eq(labels).sum().item()
        cw_total += labels.size(0)
    except Exception as e:
        print(f"Error during CW Attack evaluation: {e}")
        continue

# Calculate and display CW Attack accuracy
cw_acc = cw_correct / cw_total
print(f'CW Attack Accuracy: {cw_acc:.4f}')

# Patch Attacks: